In [1]:
## USE Python 3.7, Numpy 1.16.5, Pandas 1.1.3
import numpy as np
import pandas as pd
import itertools

In [2]:
## SET GLOBAL PARAMETERS
    # Discount rate
RATE = 0.035
    #Hyperbolic discount rate
HRATE = 0.045
    # Interest rate on investment
InvRATE = 0.035

In [3]:
##IMPORT DATA AND CREATE SERIES TO USE
# Series o       contain technical, economic, physical info pertaining to each option
# Series p       contain emission intensity pertaining to each option
# Series v       contain non-equipment subset of each option (manhours, solvent use, water use, materials etc)
# Series ext     contain socio-economic costs of environmental pollution
# Series factor  contain discount factors for standard exponential discount rates
# Series hfactor contain hyperbolic discount factors
# Series oth     contain costs over time for non-equipment subset items 

In [4]:
df = pd.read_excel(
    'Example_Option_data_ENCE.xlsx', 
    skiprows=([0, 1, 2, 3, 4])).fillna(value=0).drop(columns = 'abbreviation').set_index('O').stack()
o = df

df = (o.loc[:, ['co2', 'ch4', 'n2o', 'oth_llghg', 'so2', 'nox', 'pm2.5', 
           'oc', 'pmres', 'bc', 'nh3', 'voc', 'oth_air', 
           'ws_wat', 'oth_wat', 
           'oth_soil', 'oth_tox']
           ]).fillna(value=0)
df.index.names=['O', 'Poll']
p = df

df = pd.read_excel(
    'Example_Option_data_ENCE.xlsx', sheet_name = 'ES_Ele_emfac', 
    skiprows = ([0, 1, 2, 3, 4] + list(range(56,188)))
)
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 1'])
df = df.fillna(value = 0).set_index('T')
df = df.transpose()
df = df.stack()
df.index.names = ['Poll', 'Year']
ele_p = df

df = (o.loc[:, ['man_hours', 'solvents', 'water', 'mtrl', 'ele', 'proc_heat', 
               'coal', 'oil', 'nat_gas', 'bio_gas', 'biofuel', 'waste']
           ]).fillna(value=0)
df.index.names=['O', 'Var']
v = df

df = pd.read_excel('IN_ext_cost.xlsx', sheet_name = 'ES').set_index('Unnamed: 0').fillna(value=0)
df.index.names=['Poll']
df.columns.names=['Year']
ext = df.stack()

s = pd.Series(list(range(0,50)), index = list(range(2020,2070))).astype(np.int64)
s.index.names=['Year']
hfactor = 1/(1+HRATE*s)
factor = np.exp(-RATE*s)


df = pd.read_excel('IN_var_co.xlsx', sheet_name = 'ES').set_index('Unnamed: 0').fillna(value=0)
df.index.names=['Var']
df.columns.names=['Year']
oth = df.stack()

# writer = pd.ExcelWriter('current_test.xlsx', engine='xlsxwriter')
# ext.to_excel(writer, sheet_name='bajs')
# writer.save()

In [5]:
# Make calculation for the baseline scenario
# _bsl indicates baseline scenario (equal to s1 in IN_Scenarios.xlsx file)

## Import scenario for the first year that a given option is used or invested in
df = pd.read_excel('IN_Scenarios_ENCE.xlsx', sheet_name='Reference').set_index('Unnamed: 0')
df.index.names=['O']
df.columns.names=['Year']
sc_base = df
sc_iyr_bsl = df.fillna(0).astype('int64') #CHECK DATA TYPE HERE

### Based on the imported scenario, create technical lifetime and financial lifetime scenarios
t1 = o.loc[:, 'OT']#.astype('int64') #CHECK DATA TYPE HERE
df = pd.concat([t1, sc_base], axis = 1)

for i, j in df.iterrows():
    df1 = df.loc[i]
    df2 = df.loc[i][0].astype('int64')
    df3 = df1.fillna(method = 'ffill', limit = df2-1)
    df.loc[i] = df3

df = df.drop(columns = 0)
df = df.apply(lambda x: [y if y<=1 else 0 for y in x])
df.columns.names=['Year']
sc_OM_bsl = df
sc_iyr_bsl = sc_iyr_bsl.stack()
sc_OM_bsl = sc_OM_bsl.stack().astype('int64')

### Calculate techno-economic costs 
invest_bsl = sc_iyr_bsl * (o.loc[:, 'co_invest'] * o.loc[:, 'size'])
fixcost_bsl = sc_OM_bsl * (o.loc[:, 'co_fix_om'] * o.loc[:, 'size'])
varcost_bsl = sc_OM_bsl * (o.loc[:, 'co_var_om'] * o.loc[:, 'util'])
s = v * o.loc[:, 'util']
s = sc_OM_bsl * s
s = s * oth
s = s.groupby(['O', 'Year']).sum()
othcost_bsl = s
totcost_bsl = (invest_bsl + fixcost_bsl + varcost_bsl + othcost_bsl).fillna(0).sum(level='Year')

### Calculate emissions and external costs
s = sc_OM_bsl * p * o.loc[:, 'util']
ele_s = sc_OM_bsl * o.loc[:, 'ele'] * o.loc[:, 'util'] * ele_p
tot_s = s + ele_s
tot_s = tot_s.groupby(['Poll', 'Year']).sum()
ext_s = tot_s * ext
totext_bsl = ext_s.groupby('Year').sum()

# ### Calculate total techno-economic and environmental economic costs (societal costs)
soccost_bsl = totcost_bsl + totext_bsl

### Calculate present value of societal costs given exponential and hyperbolic discount rates
pv_totcost_bsl = (totcost_bsl * factor).sum()
hpv_totcost_bsl = (totcost_bsl * hfactor).sum()
pv_totext_bsl = (totext_bsl * factor).sum()
hpv_totext_bsl = (totext_bsl * hfactor).sum()
pv_soccost_bsl = (soccost_bsl * factor).sum()
hpv_soccost_bsl = (soccost_bsl * hfactor).sum()


In [6]:
### Make calculations for all scenarios
### Series sc_OM     contain scenario-specific use of each option
### Series sc_iyr    contain scenario-specific use of year of investment in each option

writer = pd.ExcelWriter('CBA_Output/ENCE_CBA_results.xlsx', engine='xlsxwriter')
start_row = 0
total_output = []
total_emissions = []

### Import scenarios for the first year that a given option is used or invested in
dfs = pd.read_excel('IN_Scenarios_ENCE.xlsx', sheet_name=None)
for x in dfs:
    sc = dfs[x]
    sc = sc.set_index('Unnamed: 0')
    sc.index.names=['O']
    sc.columns.names=['Year']
    sc_base = sc
    sc_iyr = sc.fillna(0).astype('int64')

### Based on the imported scenario, create technical lifetime and financial lifetime scenarios
    df = pd.concat([t1, sc_base], axis = 1)
    for i, j in df.iterrows():
        df1 = df.loc[i]
        df2 = df.loc[i][0].astype('int64')
        df3 = df1.fillna(method = 'ffill', limit = df2-1)
        df.loc[i] = df3
    df = df.drop(columns = 0)
    df = df.apply(lambda x: [y if y <=1 else 0 for y in x])
    df.columns.names = ['Year']
    sc_OM = df
    sc_iyr = sc_iyr.stack()
    sc_OM = sc_OM.stack().astype('int64')

### Calculate techno-economic costs     
    invest = sc_iyr * (o.loc[:, 'co_invest'] * o.loc[:, 'size'])
    fixcost = sc_OM * (o.loc[:, 'co_fix_om'] * o.loc[:, 'size'])
    varcost = sc_OM * (o.loc[:, 'co_var_om'] * o.loc[:, 'util'])
    s = v * o.loc[:, 'util']    
    s = sc_OM * s
    s = s * oth
    othcost = s.groupby(['O', 'Year']).sum()
    totcost = (invest + fixcost + varcost + othcost).fillna(0).sum(level='Year')

### Calculate emissions and external costs
    s = sc_OM * p * o.loc[:, 'util']
    ele_s = sc_OM * o.loc[:, 'util'] * o.loc[:, 'ele'] * ele_p
    tot_s = s + ele_s
    tot_s = tot_s.groupby(['Poll', 'Year']).sum()
    ext_s = tot_s * ext
    totext = ext_s.groupby('Year').sum()

### Calculate total techno-economic and environmental economic costs (societal costs)
    soccost = totcost + totext

### Calculate present value of societal costs given exponential and hyperbolic discount rates
    pv_totcost = (totcost * factor).sum()
    hpv_totcost = (totcost * hfactor).sum()
    pv_totext = (totext * factor).sum()
    hpv_totext = (totext * hfactor).sum()
    pv_soccost = (soccost * factor).sum()
    hpv_soccost = (soccost * hfactor).sum()

### Compare costs (technical and external) of the scenarios with the baseline (s1-scenario)
### Calculate benefit/cost ratios etc.
    pv_scen_net_welf = pv_soccost_bsl - pv_soccost # positive value = increasing welfare 
    hpv_scen_net_welf = hpv_soccost_bsl - hpv_soccost # positive = increasing welfare
    pv_scen_benefit = pv_totext_bsl - pv_totext #positive = environmental benefits
    hpv_scen_benefit = hpv_totext_bsl - hpv_totext #positive = environmental benefits
    pv_scen_cost = pv_totcost - pv_totcost_bsl # positive = increasing costs
    hpv_scen_cost = hpv_totcost - hpv_totcost_bsl # positive = increasing costs
    with np.errstate(divide='ignore', invalid='ignore'):
        bc_ratio = np.nan_to_num(pv_scen_benefit / pv_scen_cost)
        hbc_ratio = np.nan_to_num(hpv_scen_benefit / pv_scen_cost)
        hhbc_ratio = np.nan_to_num(hpv_scen_benefit / hpv_scen_cost)
    output = {'Net Welfare': pv_scen_net_welf, 'h-Net Welfare': hpv_scen_net_welf,
               'Benefits': pv_scen_benefit, 'h-Benefits': hpv_scen_benefit,
               'Costs': pv_scen_cost, 'h-Costs': hpv_scen_cost, 'b/c ratio': bc_ratio,
               'hb/c ratio': hbc_ratio, 'hb/hc ratio': hhbc_ratio}
    total_output.append(output)
    total_emissions.append(tot_s)
    
### Print to excel-file    
df = pd.DataFrame(total_output)
df2 = pd.DataFrame(total_emissions)
df.to_excel(writer, sheet_name='Economics')
df2.to_excel(writer, sheet_name='Emissions')
writer.save()